In [ ]:
建立远程仓  https://github.com/   

or create a new repository on the command line

echo "# configs" >> README.md
git init
git add README.md
git commit -m "first commit"
git remote add origin git@github.com:bruce2050/configs.git
git push -u origin master
来源： https://github.com/bruce2050/configs

按上面提示建立本地仓


git init  初始化本地仓（生成，当前目录下，生成.git命名文件）


配置：注册邮箱和登录用户
[root@acheng_host configs]# git config --global user.email "269244395@qq.com"
[root@acheng_host configs]# git config --global user.name "bruce2050"

添加远程库

首次：
git remote add origin git@github.com:michaelliao/learngit.git
                          仓名                           :用户名 /  当前目录名.git  

创建主分支
git push -u origin master

本地仓库，远程仓，主分支建立好后，在本地仓库的增删改都会被记录，可以手动或自动同步。

        1、自动实时监控和同步 ：启动wathdog，指定监控某文件，有变动则自动同步本地、远程仓库
         2、常规使用，git的命令

In [ ]:
配置自动监控某文件

在仓库下建立2个文件： 

file_sync/file_sync.py  给执行权限

file_list.txt  写入需要监控的文件，绝对路径，不一定在仓库目录下，其他目录下的可以，监控后自动加到仓库目录下

In [ ]:
#!/usr/bin/python

import sys
import time
import ntpath
import os
import re
import platform

from subprocess import call
from shutil import copy
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# git root path for files to push to remote
DIR_FOR_GIT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

# files to synchronize
SYNC_FILE_LIST = []
f = open(os.path.join(DIR_FOR_GIT, "file_list.txt"), "r")
try:
    SYNC_FILE_LIST = [line.strip() for line in f]
except Exception as e:
    raise e
finally:
    f.close()

# get filename without upper directory
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.src_path in SYNC_FILE_LIST:
            copy(event.src_path, DIR_FOR_GIT)
            cd_cmd = "cd "+DIR_FOR_GIT
            git_add_cmd = "git add -A"
            git_commit_cmd = "git commit -m " + re.escape("Update "+path_leaf(event.src_path))
            if platform.system() == "Windows":
                git_commit_cmd = "git commit -m " + re.escape("Update_"+path_leaf(event.src_path))
            git_pull_cmd = "git pull origin master"
            git_push_cmd = "git push origin master"
            call(
                cd_cmd + "&&" +
                git_add_cmd + "&&" +
                git_commit_cmd + "&&" +
                git_pull_cmd + "&&" +
                git_push_cmd,
                shell=True
            )

if __name__ == "__main__":
    observer = Observer()
    event_handler = FileChangeHandler()

    for file_path in SYNC_FILE_LIST:
        copy(file_path, DIR_FOR_GIT)
        observer.schedule(event_handler, path=os.path.dirname(os.path.realpath(file_path)), recursive=False)


    observer.start()

    try:
        while True:
            time.sleep(10)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()
              

In [1]:
!pwd

/root/python3.5.2/py3
